# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [7]:
import pandas as pd
import numpy as np
from scipy import stats

In [8]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [9]:
#Describe the data set

w = data[data.race == 'w']
b = data[data.race == 'b']

#White name callbacks summary
cb_w_tot = np.sum(w.call == 1)
tot_w_count = len(w) 
no_cb_w = tot_w_count - cb_w_tot
perc_cb_w = cb_w_tot / tot_w_count * 100

#Black name callback summary
cb_b_tot = np.sum(b.call == 1)
tot_b_count = len(b) 
no_cb_b = tot_b_count - cb_b_tot
perc_cb_b = cb_b_tot / tot_b_count * 100
perc_cb_b

data1 = {'Call': np.array([cb_w_tot, cb_b_tot, cb_w_tot + cb_b_tot]).astype(int),
        'NoCall': np.array([no_cb_w, no_cb_b, no_cb_w + no_cb_b]).astype(int),
        'Total': np.array([tot_w_count, tot_b_count, tot_w_count + tot_b_count]).astype(int),
        '%Callback': np.array([perc_cb_w, perc_cb_b, 
                           (cb_w_tot + cb_b_tot)/(tot_w_count + tot_b_count)*100]).astype(float)}

table = pd.DataFrame(data1, columns = ['Call', 'NoCall', 'Total', '%Callback'],
                    index = ['White', 'black', 'Total'], )
table

,Call,NoCall,Total,%Callback
White,235,2200,2435,9.650924
black,157,2278,2435,6.447639
Total,392,4478,4870,8.049281


### Answer 1

<p>The appropriate test for this problem is a Z score test as we are comparing the probability of call backs per race to the total probability of call backs. 
    
   The central limit theorem applies because the sample size is reasonably large.  

</p>
</div>

### Answer 2


From the summary table above, we see that the percentage of calls for interview requests is approximately 8.05%. Taking the probability of call backs for white sounding names and black sounding names we would expect one of two possibilities. 

H<sub>0</sub>: There is no significant difference in receiving interview requests between individuals with a white sounding name or black sounding name.  
- P<sub>tot</sub> 8.05% = P<sub>white</sub> 9.65% = P<sub>black</sub> 6.44%


H<sub>A</sub>: There is a significant difference in receiving interview requests between individuals with a white sounding name or black sounding name.
- P<sub>tot</sub> 8.05% != P<sub>white</sub> 9.65%  != P<sub>black</sub> 6.44%
</p>
</div>

In [11]:
#Bootstrap Method
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates


empirical_mean = np.mean(w.call) - np.mean(b.call)

#Generate shifted arrays
w_call_shifted = w.call - np.mean(w.call) + empirical_mean
b_call_shifted = b.call - np.mean(b.call) + empirical_mean

bs_replicates_w = draw_bs_reps(w_call_shifted, np.mean, 10000)
bs_replicates_b = draw_bs_reps(b_call_shifted, np.mean, 10000)

bs_replicates_calls = bs_replicates_w - bs_replicates_b
p = np.sum(bs_replicates_calls >= empirical_mean) / len(bs_replicates_calls)

#Confidence Intervals
conf_int = np.percentile(bs_replicates_calls, [2.5, 97.5])

#Margin of Error
z_crit = 1.96
var_w = np.mean(w.call)*(1-np.mean(w.call))/len(w)
var_b = np.mean(b.call)*(1-np.mean(b.call))/len(b)
sem = np.sqrt(var_w + var_b) * z_crit

print('p = ', p)
print(sem)
print('95% Confidence Interval =', conf_int) 


0.032032855
p =  0.0
0.015255406348684322
95% Confidence Interval = [-0.01519507  0.01519507]


In [71]:
#Frequentist Approach
p_w = np.mean(w.call)
p_b = np.mean(b.call)

diff_mean = p_w - p_b

#Calculate the variance of Pw - Pb and std deviation
w_var = p_w * (1 - p_w) / len(w)
b_var = p_b * (1 - p_b) / len(b)
tot_var = w_var + b_var
tot_std = np.sqrt(tot_var)

#Calculate margin of error with alpha = 0.05
z_crit = 1.96 
sem_tot = z_crit * tot_std
max_int = diff_mean + sem_tot
min_int = diff_mean - sem_tot

print("Margin of Error = ", sem_tot)
print('95% Confidence Interval:', '[', min_int,',', max_int, ']')

#calculate z value
z = (diff_mean - 0) / tot_std
print('z =', z)

Margin of Error =  0.015255406348684322
95% Confidence Interval: [ 0.016777448506376254 , 0.0472882612037449 ]
z = 4.115550519002299


### Answer 3

From the bootstrap approach, we reject the null hypothesis with a 0.0 p value. There is a significant difference between white sounding name call backs and black sounding names call backs. Our 95% confidence interval is between [-0.01519507, 0.01560575] showing a call back percentage difference within that interval is not abnormal or significant. A difference of 3.61 is well out of the range therefore showing the results are significantly abnormal. 


The results of the frequentist approach are similar to our bootstrap approach. At alpha = 0.05 we expect a z value of 1.96. Our results have a z value of 4.12, well beyond the critcal z value therefore we reject the null hypothesis. The results of our analysis show significant difference between the call backs of each race. From the frequentist approach we expect a 95% confidence interval between [0.017, 0.047], our difference of 3.61 is out of the range, and is therefore abnormal. 


### Answer 4

<p> From the data obtained, there is a significant difference between call backs received by white sounding names and black sounding names. White sounding names have recieved about 59% of the interview requests. 

</p>
</div>

### Answer 5

From our analysis all we can say with 100% confidence is that there is a significant difference between interview requests for white sounding names and black sounding names. However, we cannot say with 100% certainty that race is the most important variable in interview decisions. There are other factors that need to be taken into consideration which could affect the results of the race variable.

- When was the position posted and whether or not a candidate was already hired when applications were received. 
- What areas were the jobs located and what sort of racial diversity is in that location. 
- What sort of qualifications was each job looking for? Were the mock resumes over qualified or underqualified?

A follow up research experiment would be to isolate the results to one company and one available position to see the number of interview requests for the same resumes with different names. Thus we would isolate the variable down to only one factor, race.